# HGPS Jupyter notebook


The H.E.S.S. Galactic Plane Survey (HGPS) is the first deep and wide survey of the Milky Way in TeV gamma-rays.

This is a Jupyter notebook that illustrates how to work with the HGPS data from Python.

The release data consistes of survey maps and a source catalog

H.E.S.S. Galactic Plane Survey 

https://www.mpi-hd.mpg.de/hfm/HESS/hgps

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from astropy.io import fits
from astropy.table import Table

## Maps

This section shows you how to ... <TODO>

In [3]:
from gammapy.maps import Map

In [4]:
survey_map = Map.read('../build/data/hgps_map_significance_0.1deg_v1.fits.gz')

In [5]:
survey_map.get_by_coords((0, 0))

array([73.72828], dtype=float32)

## Catalog

In [6]:
from gammapy.catalog import SourceCatalogHGPS

In [7]:
cat = SourceCatalogHGPS()

In [8]:
source = cat[0]

In [9]:
print(source)


*** Basic info ***

Catalog row index (zero-based) : 0
Source name          : HESS J0835-455
Analysis reference   : HGPS
Source class         : PWN
Identified object    : Vela X
Gamma-Cat id         : 37


*** Info from map analysis ***

RA                   :  128.887 deg = 8h35m33s
DEC                  :  -45.659 deg = -45d39m32s
GLON                 :  263.960 +/- 0.025 deg
GLAT                 :   -3.048 +/- 0.027 deg
Position Error (68%) : 0.057 deg
Position Error (95%) : 0.093 deg
ROI number           : 18
Spatial model        : 3-Gaussian
Spatial components   : HGPSC 001, HGPSC 002, HGPSC 003
TS                   : 1553.2
sqrt(TS)             : 39.4
Size                 : 0.585 +/- 0.052 (UL: nan) deg
R70                  : 0.887 deg
RSpec                : 0.500 deg
Total model excess   : 8781.6
Excess in RSpec      : 3580.1
Model Excess in RSpec : 3637.4
Background in RSpec  : 5936.9
Livetime             : 64.9 hours
Energy threshold     : 0.47 TeV
Source flux (>1 TeV) : (15.3

In [10]:
print(source.spectral_model)

ExponentialCutoffPowerLaw

Parameters: 

	   name     value     error         unit      min max frozen
	--------- --------- --------- --------------- --- --- ------
	    index 1.354e+00 7.733e-02                 nan nan  False
	amplitude 6.408e-12 3.260e-13 1 / (cm2 s TeV) nan nan  False
	reference 1.697e+00 0.000e+00             TeV nan nan   True
	  lambda_ 8.152e-02 1.154e-02         1 / TeV nan nan  False

Covariance: 

	name/name  index  amplitude lambda_ 
	--------- ------- --------- --------
	    index 0.00598       0.0      0.0
	amplitude     0.0  1.06e-25      0.0
	  lambda_     0.0       0.0 0.000133


In [11]:
source.flux_points.table

e_ref,e_min,e_max,dnde,dnde_errn,dnde_errp,dnde_ul,is_ul
TeV,TeV,TeV,1 / (cm2 s TeV),1 / (cm2 s TeV),1 / (cm2 s TeV),1 / (cm2 s TeV),
float32,float32,float32,float32,float32,float32,float32,bool
0.4216965,0.28729847,0.6189658,6.5541766e-11,1.0499469e-11,1.037914e-11,8.636929e-11,False
0.9531619,0.6189658,1.4677993,1.1986031e-11,1.2919824e-12,1.2886871e-12,1.4526381e-11,False
2.260303,1.4677993,3.4807005,3.52555e-12,2.3576906e-13,2.4082174e-13,4.0178963e-12,False
5.360023,3.4807005,8.254042,8.324112e-13,5.495922e-14,5.6203943e-14,9.495746e-13,False
12.710618,8.254042,19.573418,1.5962396e-13,1.3232291e-14,1.3493189e-14,1.8696622e-13,False
30.141624,19.573418,46.41589,1.11618885e-14,2.1889015e-15,2.3228553e-15,1.5994852e-14,False
